In [1]:
#all imports to get this done
import json
import os
import pandas as pd
from bs4 import BeautifulSoup
#get the file cracked open
with open(r'C:\Users\monae\Downloads\from-the-lemon-trees.ghost.2025-01-24-21-57-30.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

Issues: I need to go through a lot
db is a list, 
that list is holding a dictionary ('0')
that dictionary ('0') is hodling another dictionary called 'data'
this dictionary ('data') is holding a the list of posts that I want

Even then what I want is a series of dictionaries, each representing a single post

Issue: this is a list of dictionaries
data['db'] is a list
data['db'][0] is a dictionary
data['db'][0]['data'] is a dictionary
data['db'][0]['data']['posts'] is a list
data['db'][0]['data']['posts'][0] is a dictionary -> level from which I need to acquire information

Information to get: id', 'title', 'plaintext', 'status', 'published_at'


Eventually, I also want from the 'data' dictionary the post tags and tags (both of which are dictionaries) to join to a flattened dataframe so I can also have how they were labeled

In [2]:
#glancing through the data to make sure I know what type I land in
allPosts = data['db'][0]['data']['posts'][0]
type(allPosts)
#print(all_posts)

dict

In [3]:
#supporting dataframes to show where we land and continue transforming
bigFrame = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data['db'][0].items()]))
bigFrame
normFrame = pd.json_normalize(json.loads(bigFrame.to_json(orient='records')))
normFrame = pd.json_normalize(json.loads(normFrame['data'].to_json(orient='index')))
normFrame = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in normFrame.items()])).T

#single post acquired
#normbigframe3[0][6][0] is depreciating so make sure to get used to iloc
normFrame.iloc[6,0][3]

{'id': '622f8cc77d710c003dadf5bb',
 'uuid': 'cfa4429a-704c-4063-acd1-e092a468c3f5',
 'title': 'Resources',
 'slug': 'resources',
 'mobiledoc': None,
 'html': '<p>Coming soon!</p>',
 'comment_id': '622f8cc77d710c003dadf5bb',
 'plaintext': 'Coming soon!',
 'feature_image': None,
 'featured': 0,
 'type': 'page',
 'status': 'draft',
 'locale': None,
 'visibility': 'public',
 'email_recipient_filter': 'all',
 'created_at': '2022-03-14T18:43:19.000Z',
 'updated_at': '2024-06-05T23:46:26.000Z',
 'published_at': '2022-03-14T18:47:13.000Z',
 'custom_excerpt': None,
 'codeinjection_head': None,
 'codeinjection_foot': None,
 'custom_template': None,
 'canonical_url': None,
 'newsletter_id': None,
 'lexical': '{"root":{"children":[{"children":[{"detail":0,"format":0,"mode":"normal","style":"","text":"Coming soon!","type":"extended-text","version":1}],"direction":"ltr","format":"","indent":0,"type":"paragraph","version":1}],"direction":"ltr","format":"","indent":0,"type":"root","version":1}}',
 'sh

From post tags portion, I need post_id and tag_id so I can join later
for post_tags 
for tagging, join on tag_id and id

In [4]:
#post details
postDetailsDf = pd.DataFrame.from_dict(normFrame.iloc[6,0])
postDetailsDf.head(4)

,id,uuid,title,slug,mobiledoc,html,comment_id,plaintext,feature_image,featured,...,updated_at,published_at,custom_excerpt,codeinjection_head,codeinjection_foot,custom_template,canonical_url,newsletter_id,lexical,show_title_and_feature_image
0,61f0be63f6b6e5003bbb257c,e36559fc-8ec4-4a49-bf6b-b7cefaeb1f26,About,about,None,"<p>I'm Lemon (she/they), a medicine maker, pra...",61f0be63f6b6e5003bbb257c,"I'm Lemon (she/they), a medicine maker, practi...",__GHOST_URL__/content/images/2022/01/beach-pho...,0,...,2024-03-14T03:29:27.000Z,2022-01-26T04:35:47.000Z,None,None,None,None,None,None,"{""root"":{""children"":[{""children"":[{""detail"":0,...",1
1,61f0d04af6b6e5003bbb25b6,42f591ff-714c-44da-bc98-18ae9df476ce,"Femininity, I guess?",femininity-i-guess,None,<p>Woman: an adult female human being; a perso...,61f0d04af6b6e5003bbb25b6,Woman: an adult female human being; a person w...,None,0,...,2024-03-14T03:37:10.000Z,None,None,None,None,None,None,None,"{""root"":{""children"":[{""children"":[{""detail"":0,...",1
2,62252a75fb4b09003dd923f8,b852cc6e-f9b2-4da4-ab47-61dcaa2967a7,Things I wish I knew...,things-i-wish-i-knew,None,<ul><li>Shit will be hard. And sometimes for n...,62252a75fb4b09003dd923f8,* Shit will be hard. And sometimes for no cle...,None,0,...,2023-11-16T02:13:35.000Z,2022-03-06T21:41:48.000Z,None,None,None,None,None,None,"{""root"":{""children"":[{""children"":[{""children"":...",1
3,622f8cc77d710c003dadf5bb,cfa4429a-704c-4063-acd1-e092a468c3f5,Resources,resources,None,<p>Coming soon!</p>,622f8cc77d710c003dadf5bb,Coming soon!,None,0,...,2024-06-05T23:46:26.000Z,2022-03-14T18:47:13.000Z,None,None,None,None,None,None,"{""root"":{""children"":[{""children"":[{""detail"":0,...",1


In [5]:
postDetailsDf.drop(postDetailsDf.columns.difference(['id', 'title', 'published_at', 'html', 'plaintext']), axis=1, inplace=True)
postDetailsDf.head(4)

,id,title,html,plaintext,published_at
0,61f0be63f6b6e5003bbb257c,About,"<p>I'm Lemon (she/they), a medicine maker, pra...","I'm Lemon (she/they), a medicine maker, practi...",2022-01-26T04:35:47.000Z
1,61f0d04af6b6e5003bbb25b6,"Femininity, I guess?",<p>Woman: an adult female human being; a perso...,Woman: an adult female human being; a person w...,None
2,62252a75fb4b09003dd923f8,Things I wish I knew...,<ul><li>Shit will be hard. And sometimes for n...,* Shit will be hard. And sometimes for no cle...,2022-03-06T21:41:48.000Z
3,622f8cc77d710c003dadf5bb,Resources,<p>Coming soon!</p>,Coming soon!,2022-03-14T18:47:13.000Z


In [6]:
#make sure I have the correct rows to break out from the greater frame
bigFrame.iloc[[10,19]]
#posts_tags broken out
postTags = bigFrame.iloc[10,1]
postTagsDf = pd.DataFrame.from_dict(postTags, orient='columns')
#still need post_id for later join, 
postTagsDf.drop(['id', 'sort_order'], axis = 1, inplace=True)
postTagsDf.head()

,post_id,tag_id
0,62315d9f7d710c003dadf625,624bb0a5eb31aa004d9e45b2
1,624b7b59eb31aa004d9e44b8,624bb531eb31aa004d9e45bf
2,62315f6d7d710c003dadf637,624bb0a5eb31aa004d9e45b2
3,623162ba7d710c003dadf689,624bb0a5eb31aa004d9e45b2
4,624b8915eb31aa004d9e4517,624bb0a5eb31aa004d9e45b2


In [7]:
#tags broken out
tags = bigFrame.iloc[19,1]
tagsDf = pd.DataFrame.from_dict(tags, orient='columns')
#dropping literally everything else because don't need it
tagsDf.drop(tagsDf.columns.difference(['id', 'name', 'description']), axis=1, inplace=True)
#no need for head here since there's so little in terms of tagging
tagsDf

,id,name,description
0,624bb0a5eb31aa004d9e45b2,Collection,None
1,624bb531eb31aa004d9e45bf,Tips & Tricks,None
2,62c0ad9a14c349003d9e5843,Reflections,thoughts and lessons learned
3,62c0afab14c349003d9e584d,Products & Services,medicine for the soul
4,62cba5441bf7c8004da12628,Poetry,a reservoir of my deepest personal thoughts an...


In [8]:
#left join on post_tagsdf['tag_id'] and tagsdf['id']
postTagsDf = pd.merge(postTagsDf, tagsDf, left_on='tag_id', right_on='id', how='left')
postTagsDf
#now merged, going to drop the now unnecessary and redundant columns
postTagsDf.drop(['tag_id', 'id'], axis=1, inplace= True)
postTagsDf.head()

,post_id,name,description
0,62315d9f7d710c003dadf625,Collection,None
1,624b7b59eb31aa004d9e44b8,Tips & Tricks,None
2,62315f6d7d710c003dadf637,Collection,None
3,623162ba7d710c003dadf689,Collection,None
4,624b8915eb31aa004d9e4517,Collection,None


In [9]:
#merge the last dfs to what I need them to be
finalDf = pd.merge(pd.DataFrame.from_dict(postDetailsDf, orient='columns'), postTagsDf, how="outer",left_on='id', right_on='post_id')
#remove the extra id column that was joined on
finalDf.drop(['post_id'], axis=1,inplace=True)
finalDf.head(4)

,id,title,html,plaintext,published_at,name,description
0,61f0be63f6b6e5003bbb257c,About,"<p>I'm Lemon (she/they), a medicine maker, pra...","I'm Lemon (she/they), a medicine maker, practi...",2022-01-26T04:35:47.000Z,NaN,NaN
1,61f0d04af6b6e5003bbb25b6,"Femininity, I guess?",<p>Woman: an adult female human being; a perso...,Woman: an adult female human being; a person w...,None,NaN,NaN
2,62252a75fb4b09003dd923f8,Things I wish I knew...,<ul><li>Shit will be hard. And sometimes for n...,* Shit will be hard. And sometimes for no cle...,2022-03-06T21:41:48.000Z,NaN,NaN
3,622f8cc77d710c003dadf5bb,Resources,<p>Coming soon!</p>,Coming soon!,2022-03-14T18:47:13.000Z,NaN,NaN


In [10]:
finalDf['published_at'] = pd.to_datetime(finalDf['published_at'])
finalDf.sort_values(by='published_at', ascending=True, inplace=True)
finalDf.head(3)

,id,title,html,plaintext,published_at,name,description
0,61f0be63f6b6e5003bbb257c,About,"<p>I'm Lemon (she/they), a medicine maker, pra...","I'm Lemon (she/they), a medicine maker, practi...",2022-01-26 04:35:47+00:00,NaN,NaN
2,62252a75fb4b09003dd923f8,Things I wish I knew...,<ul><li>Shit will be hard. And sometimes for n...,* Shit will be hard. And sometimes for no cle...,2022-03-06 21:41:48+00:00,NaN,NaN
3,622f8cc77d710c003dadf5bb,Resources,<p>Coming soon!</p>,Coming soon!,2022-03-14 18:47:13+00:00,NaN,NaN


In [11]:
def neat_function(i):
    print(f'{finalDf['title'][i]}')
    try:
        print(f'Originally published to www.fromthelemontrees.com on {finalDf['published_at'][i].strftime('%m.%d.%Y')}\n\n')
    except:
        print("time unknown or unpublished")
    try:
        print(BeautifulSoup(finalDf['plaintext'][i], 'html.parser').prettify())
    except:
        print("")
    print(f'{finalDf['name'][i]}')
    print('\n\n')
neat_function(50)

mint sings in the morning
Originally published to www.fromthelemontrees.com on 03.19.2024


before the spring comes, we clean

spring cleaning, we call it



maybe it is long overdue

that I air out my life

shaking loose the limp and lifeless

wishing wellness for having moved from me

letting go



open my doors for another chapter to begin

even if that means without my beloveds



it's just that time



to stand

to move

to cleanse



mint has warmed the way

the spirit of refreshment has come

and with her I will leave

Poetry





In [ ]:
#loop through df column
publishing = []
for index, row in finalDf.iterrows():
    publishing.append(neat_function(index))

Cell outputs were cleared, as to provide some additional privacy, but also because the output is much too long.

We also need to capture the drafts as she still intends to work on these posts and articles.

In [13]:
nullPosts = postDetailsDf[postDetailsDf['published_at'].isnull()]
nullPosts.tail(2)

,id,title,html,plaintext,published_at
45,660f20af2a162e0001cb584a,The Art of Experimentation,"<p>There we go, a sufficiently pretentious tit...","There we go, a sufficiently pretentious title ...",None
46,660f22292a162e0001cb584e,Orange and Gladness,None,None,None


In [ ]:
publishingnull = []
for index, row in nullPosts.iterrows():
    publishingnull.append(neat_function(index))

Same deal on the cell outputs.

This is as far as they wanted processed and that means my work here is done! Glad they're happy with it.